# Armoria API

In [15]:
import requests
import json
from PIL import Image
from io import BytesIO

class ArmoriaAPIWrapper:
    def __init__(self, size, format, coa):
        self.format = format
        payload = {"size": size, 
                   "format": format,
                   "coa": json.dumps(coa)
                  }
        self.r = requests.get('https://armoria.herokuapp.com/', params=payload)
#         print(self.r.url)
    
    def get_image_bytes(self):
        i = Image.open(BytesIO(self.r.content))
        return i
    
    def show_image(self):
        i = self.get_image_bytes()
        i.show()
    
    def save_image(self, full_path):
        i = self.get_image_bytes()    
        i.save(full_path, 'png')



In [16]:
coa_lion = {"t1":"or",
       "shield":"heater",
       "charges":[{"charge":"lionRampant",
                   "t":"gules",
                   "p":"e",
                   "size":1.5}]
      }

api = ArmoriaAPIWrapper(size = 500,format="png", coa=coa_lion)
api.show_image()

coa_cross = {"t1":"argent",
       "shield":"heater",
       "charges":[{"charge":"crossMoline",
                   "t":"azure",
                   "p":"e",
                   "size":1.5}]
      }

api = ArmoriaAPIWrapper(size = 500,format="png", coa=coa_cross)
api.show_image()


# Map our dataset to the API keywords

In [17]:
LION_MODIFIERS_MAP = {
    'lion rampant': 'lionRampant',
    'lion passt': 'lionPassant',
    'lion passt guard': 'lionPassantGuardant',
    "lion's head": 'lionHeadCaboshed'
}

CROSS_MODIFIERS_MAP = {
    'cross': 'crossHummetty' ,
    'cross moline': 'crossMoline',
    'cross patonce': 'crossPatonce',
}

    
EAGLE_MODIFIERS_MAP = {
    'eagle': 'eagle' ,
    'eagle doubleheaded': 'eagleTwoHeards',
}


MODIFIERS_MAP = {**LION_MODIFIERS_MAP, **CROSS_MODIFIERS_MAP, **EAGLE_MODIFIERS_MAP}

COLORS_MAP = { 'A': 'argent', # silver
              'B': 'azure', # blue
              'O': 'or', # gold 
              'S': 'sable', # black
              'G': 'gules', # red
              'V': 'vert' # green
             }
#               'E':'',
#               'X': '', 
#               'Z': ''}


# Armoria-API = possible values for single position of a charge in an image
p = ['a','b','c','d','e','f','g','h','i','y','z']

# Armoria-API = possible values for a charge scale 
# note that for large charges in a side positions means that a part of the charge is out of the shield
scale = ['0.5','1','1.5']

In [9]:
possible_permutations = list(dict.fromkeys(itertools.product(list(all_possible_permutations), p, scale)))
print('Total number of permutations with different position and scale:', len(possible_permutations))


Total number of permutations with different position and scale: 10692


# Generate Permutations

In [18]:
import itertools
import src.alphabet as alphabet

all_possible_permutations = []

# lion, modifiers and colors
permutations1 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( LION_MODIFIERS_MAP.keys()))))

# cross, modifiers and colors
permutations2 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()))))

# eagle, modifiers and colors
permutations3 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( EAGLE_MODIFIERS_MAP.keys()))))

all_possible_permutations = permutations1 + permutations2 + permutations3

print('Total number of permutations:', len(all_possible_permutations))
all_possible_permutations

Total number of permutations: 324


[('A', 'A', 'lion rampant'),
 ('A', 'A', 'lion passt'),
 ('A', 'A', 'lion passt guard'),
 ('A', 'A', "lion's head"),
 ('A', 'B', 'lion rampant'),
 ('A', 'B', 'lion passt'),
 ('A', 'B', 'lion passt guard'),
 ('A', 'B', "lion's head"),
 ('A', 'O', 'lion rampant'),
 ('A', 'O', 'lion passt'),
 ('A', 'O', 'lion passt guard'),
 ('A', 'O', "lion's head"),
 ('A', 'S', 'lion rampant'),
 ('A', 'S', 'lion passt'),
 ('A', 'S', 'lion passt guard'),
 ('A', 'S', "lion's head"),
 ('A', 'G', 'lion rampant'),
 ('A', 'G', 'lion passt'),
 ('A', 'G', 'lion passt guard'),
 ('A', 'G', "lion's head"),
 ('A', 'V', 'lion rampant'),
 ('A', 'V', 'lion passt'),
 ('A', 'V', 'lion passt guard'),
 ('A', 'V', "lion's head"),
 ('B', 'A', 'lion rampant'),
 ('B', 'A', 'lion passt'),
 ('B', 'A', 'lion passt guard'),
 ('B', 'A', "lion's head"),
 ('B', 'B', 'lion rampant'),
 ('B', 'B', 'lion passt'),
 ('B', 'B', 'lion passt guard'),
 ('B', 'B', "lion's head"),
 ('B', 'O', 'lion rampant'),
 ('B', 'O', 'lion passt'),
 ('B', '

In [12]:
def get_armoria_api_payload(label):
    shield_color = label[0]
    charge_color = label[1]
    charge = label[2]
    
    api_shield_color = COLORS_MAP[shield_color]
    api_charge_color = COLORS_MAP[charge_color]
    api_charge = MODIFIERS_MAP[charge]

    coa = {"t1": api_shield_color, 
       "shield":"heater",
       "charges":[{"charge":api_charge,
                   "t": api_charge_color,
                   "p":"e", # shuffel
                   "size":1.5}] # shuffel
      }

    return coa

def save_label_txt(file_name, label):
    f = open(file_name ,"w+")
    f.write(' '.join(label))
    f.close()


def generate_dataset(folder_name, permutations):
    counter = 1

    # create a file to store all generated images files names and labels
    f = open(folder_name + '/' + 'captions.txt' ,"w+")
    f.write('image,caption')
    f.write('\n')

    for label in all_possible_permutations:

        payload = get_armoria_api_payload(label)

        api = ArmoriaAPIWrapper(size = 500,format="png", coa=payload)

        sample_name = 'image_' + str(counter)

        image_full_path = folder_name + '/images/' + sample_name + '.png'
        api.save_image(image_full_path)

    #     caption_full_path = FOLDER_NAME + '/images/' + sample_name + ".txt"
    #     save_label_txt(caption_full_path, label)

        text_label = ' '.join(label)
        print('Image "{}" for label "{}" has been generated succfully' .format(image_full_path, text_label) )

        counter = counter + 1

        ## store image labels in on txt file (needed for data loader) 
        f.write(sample_name + '.png,' + text_label)
        f.write('\n')

    f.close()


In [23]:
FOLDER_NAME = 'generated-data-api'
generate_dataset(FOLDER_NAME, all_possible_permutations)

Image "generated-data-api/images/image_1.png" for label "A A lion rampant" has been generated succfully
Image "generated-data-api/images/image_2.png" for label "A A lion passt" has been generated succfully
Image "generated-data-api/images/image_3.png" for label "A A lion passt guard" has been generated succfully
Image "generated-data-api/images/image_4.png" for label "A A lion's head" has been generated succfully
Image "generated-data-api/images/image_5.png" for label "A B lion rampant" has been generated succfully
Image "generated-data-api/images/image_6.png" for label "A B lion passt" has been generated succfully
Image "generated-data-api/images/image_7.png" for label "A B lion passt guard" has been generated succfully
Image "generated-data-api/images/image_8.png" for label "A B lion's head" has been generated succfully
Image "generated-data-api/images/image_9.png" for label "A O lion rampant" has been generated succfully
Image "generated-data-api/images/image_10.png" for label "A O 

Image "generated-data-api/images/image_79.png" for label "S B lion passt guard" has been generated succfully
Image "generated-data-api/images/image_80.png" for label "S B lion's head" has been generated succfully
Image "generated-data-api/images/image_81.png" for label "S O lion rampant" has been generated succfully
Image "generated-data-api/images/image_82.png" for label "S O lion passt" has been generated succfully
Image "generated-data-api/images/image_83.png" for label "S O lion passt guard" has been generated succfully
Image "generated-data-api/images/image_84.png" for label "S O lion's head" has been generated succfully
Image "generated-data-api/images/image_85.png" for label "S S lion rampant" has been generated succfully
Image "generated-data-api/images/image_86.png" for label "S S lion passt" has been generated succfully
Image "generated-data-api/images/image_87.png" for label "S S lion passt guard" has been generated succfully
Image "generated-data-api/images/image_88.png" fo

Image "generated-data-api/images/image_157.png" for label "A G cross" has been generated succfully
Image "generated-data-api/images/image_158.png" for label "A G cross moline" has been generated succfully
Image "generated-data-api/images/image_159.png" for label "A G cross patonce" has been generated succfully
Image "generated-data-api/images/image_160.png" for label "A V cross" has been generated succfully
Image "generated-data-api/images/image_161.png" for label "A V cross moline" has been generated succfully
Image "generated-data-api/images/image_162.png" for label "A V cross patonce" has been generated succfully
Image "generated-data-api/images/image_163.png" for label "B A cross" has been generated succfully
Image "generated-data-api/images/image_164.png" for label "B A cross moline" has been generated succfully
Image "generated-data-api/images/image_165.png" for label "B A cross patonce" has been generated succfully
Image "generated-data-api/images/image_166.png" for label "B B c

Image "generated-data-api/images/image_236.png" for label "V A cross moline" has been generated succfully
Image "generated-data-api/images/image_237.png" for label "V A cross patonce" has been generated succfully
Image "generated-data-api/images/image_238.png" for label "V B cross" has been generated succfully
Image "generated-data-api/images/image_239.png" for label "V B cross moline" has been generated succfully
Image "generated-data-api/images/image_240.png" for label "V B cross patonce" has been generated succfully
Image "generated-data-api/images/image_241.png" for label "V O cross" has been generated succfully
Image "generated-data-api/images/image_242.png" for label "V O cross moline" has been generated succfully
Image "generated-data-api/images/image_243.png" for label "V O cross patonce" has been generated succfully
Image "generated-data-api/images/image_244.png" for label "V S cross" has been generated succfully
Image "generated-data-api/images/image_245.png" for label "V S c

Image "generated-data-api/images/image_314.png" for label "V A eagle doubleheaded" has been generated succfully
Image "generated-data-api/images/image_315.png" for label "V B eagle" has been generated succfully
Image "generated-data-api/images/image_316.png" for label "V B eagle doubleheaded" has been generated succfully
Image "generated-data-api/images/image_317.png" for label "V O eagle" has been generated succfully
Image "generated-data-api/images/image_318.png" for label "V O eagle doubleheaded" has been generated succfully
Image "generated-data-api/images/image_319.png" for label "V S eagle" has been generated succfully
Image "generated-data-api/images/image_320.png" for label "V S eagle doubleheaded" has been generated succfully
Image "generated-data-api/images/image_321.png" for label "V G eagle" has been generated succfully
Image "generated-data-api/images/image_322.png" for label "V G eagle doubleheaded" has been generated succfully
Image "generated-data-api/images/image_323.p

In [13]:
# Generate smaller dataset for local testing

import itertools
import src.alphabet as alphabet

all_possible_permutations = []

# lion, modifiers and colors
permutations1 = list(dict.fromkeys(itertools.product(['A'],list(COLORS_MAP.keys()), list( LION_MODIFIERS_MAP.keys()))))

# cross, modifiers and colors
permutations2 = list(dict.fromkeys(itertools.product(['A'], list(COLORS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()))))

# eagle, modifiers and colors
permutations3 = list(dict.fromkeys(itertools.product(['A'], list(COLORS_MAP.keys()), list( EAGLE_MODIFIERS_MAP.keys()))))

all_possible_permutations = permutations1 + permutations2 + permutations3

print('Total number of permutations:', len(all_possible_permutations))
all_possible_permutations

Total number of permutations: 54


[('A', 'A', 'lion rampant'),
 ('A', 'A', 'lion passt'),
 ('A', 'A', 'lion passt guard'),
 ('A', 'A', "lion's head"),
 ('A', 'B', 'lion rampant'),
 ('A', 'B', 'lion passt'),
 ('A', 'B', 'lion passt guard'),
 ('A', 'B', "lion's head"),
 ('A', 'O', 'lion rampant'),
 ('A', 'O', 'lion passt'),
 ('A', 'O', 'lion passt guard'),
 ('A', 'O', "lion's head"),
 ('A', 'S', 'lion rampant'),
 ('A', 'S', 'lion passt'),
 ('A', 'S', 'lion passt guard'),
 ('A', 'S', "lion's head"),
 ('A', 'G', 'lion rampant'),
 ('A', 'G', 'lion passt'),
 ('A', 'G', 'lion passt guard'),
 ('A', 'G', "lion's head"),
 ('A', 'V', 'lion rampant'),
 ('A', 'V', 'lion passt'),
 ('A', 'V', 'lion passt guard'),
 ('A', 'V', "lion's head"),
 ('A', 'A', 'cross'),
 ('A', 'A', 'cross moline'),
 ('A', 'A', 'cross patonce'),
 ('A', 'B', 'cross'),
 ('A', 'B', 'cross moline'),
 ('A', 'B', 'cross patonce'),
 ('A', 'O', 'cross'),
 ('A', 'O', 'cross moline'),
 ('A', 'O', 'cross patonce'),
 ('A', 'S', 'cross'),
 ('A', 'S', 'cross moline'),
 ('A

In [15]:
FOLDER_NAME = 'generated-data-api-small'
generate_dataset(FOLDER_NAME, all_possible_permutations)

Image "generated-data-api-small/images/image_1.png" for label "A A lion rampant" has been generated succfully
Image "generated-data-api-small/images/image_2.png" for label "A A lion passt" has been generated succfully
Image "generated-data-api-small/images/image_3.png" for label "A A lion passt guard" has been generated succfully
Image "generated-data-api-small/images/image_4.png" for label "A A lion's head" has been generated succfully
Image "generated-data-api-small/images/image_5.png" for label "A B lion rampant" has been generated succfully
Image "generated-data-api-small/images/image_6.png" for label "A B lion passt" has been generated succfully
Image "generated-data-api-small/images/image_7.png" for label "A B lion passt guard" has been generated succfully
Image "generated-data-api-small/images/image_8.png" for label "A B lion's head" has been generated succfully
Image "generated-data-api-small/images/image_9.png" for label "A O lion rampant" has been generated succfully
Image "g